# Fine-Tuning Transformer Models for Classification of Digital Behavioural Data
## Classfying political stance

By Indira Sen

This notebook will demonstrate how to fine-tune transformer models like BERT for classification. We show two options for using transformer models in Python
- HuggingFace `transformers`
- Simple Transformers

We will fine-tune transformer models like BERT, DistilBERT, and RoBERTa for a task common in Computational Social Science: Political Stance Detection


<br><br>

## **Import necessary Python libraries and modules**

First, we will import necessary Python libraries and modules. These include scikit-learn (`sklearn`) and PyTorch (`torch`), for various machine learning tools.

In [1]:
# Basic Python modules
from collections import defaultdict

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch

We first download the datasets we need for finetuning our models. This is a **supervised** classification task, therefore, we will need labeled data. We download the following datasets:

1. PStance [https://github.com/chuchun8/PStance]

In [2]:
data = pd.read_csv('raw_train_biden.csv')
data.head()

,Tweet,Target,Stance
0,Joe Biden is looking to gather votes from unsu...,Joe Biden,AGAINST
1,Check out the latest podcast conversation betw...,Joe Biden,FAVOR
2,Thank you Secretary Clinton for your endorseme...,Joe Biden,FAVOR
3,Happening now: @JoeBiden kicking off #Hispanic...,Joe Biden,FAVOR
4,Thank you Mayor @KeishaBottoms for opening our...,Joe Biden,FAVOR


We first use the [`simpletransformers`](https://simpletransformers.ai/) package which is more beginner-friendly

In [3]:
! pip3 install simpletransformers

The basic steps for finetuning a classifier using simpletrasnformers are:
- Initialize a model based on a specific architechture (BERT, DistilBERT, etc)
- Train the model with train_model()
- Evaluate the model with eval_model()
- Make predictions on (unlabelled) data with predict()

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [5]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

We need to preprocess the data first before we start the finetuning process. In this step, we split the dataset into **train** and **test** sets to have a fully held-out test set that can be used to evaluate our classifier.

We can also create a **validation** that is used during the fine tuning process for hyperparameter tuning, but that is not mandatory.

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data, stratify=data['Stance'], test_size=0.2)

We now convert the dataframes into a format that can be read by simpletransformers. This is a dataframe with the columns 'text' and 'labels'. The 'labels' column should be numerical, so we use **one-hot encoding** to transform our string stance labels to numerical ones.

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_df['Stance'])
train_df['labels'] = le.transform(train_df['Stance'])
test_df['labels'] = le.transform(test_df['Stance'])

In [8]:
# to see which number was mapped to which class:
list(le.inverse_transform([0,1]))

['AGAINST', 'FAVOR']

So, 0 is 'against' and 1 is 'favor'. We now have the appropriate data structure. The next step is setting the training parameters and loading the classification model, in this case, DistilBERT, a lightweight model that can be trained relatively quickly compared to other transformer variants like BERT and RoBERTa.

For training parameters, we have many to choose from such as the learning rate, whether we want to stop early or not, where we should save the model, and more. You can find all of them here: https://simpletransformers.ai/docs/usage/

As a minimal setup, we will just set the number of **epochs**, i.e., the number of passes the model does over the full training set. For recent transformer models, epochs are usually set to 2 or 3, after which overfitting may happen.

In [9]:

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=3, output_dir='output_st', overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel(
    "distilbert", "distilbert-base-uncased", args=model_args
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We are now finally ready to begin training! This might take a while, especially when we're not using a GPU.

In [10]:
train_df = train_df[['Tweet', 'labels']]
test_df = test_df[['Tweet', 'labels']]

In [11]:
len(train_df)

4644

In [12]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4644 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/581 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/581 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/581 [00:00<?, ?it/s]

(1743, 0.3908739920942796)

After training our model, we can use it to make predictions for unlabeled datapoints to classify the stance of the tweet towards the predefined target.

In [13]:
anti_biden_tweet = "Ugh, this was true yesterday and it's also true now: Biden is an idiot"
predictions, raw_outputs = model.predict([anti_biden_tweet])
le.inverse_transform(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array(['AGAINST'], dtype=object)

We can also use the held-out test set to quantitatively evaluate our model.

In [14]:
len(test_df) / 8

145.25

In [15]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1162 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/146 [00:00<?, ?it/s]

{'mcc': 0.519245155677564,
 'tp': 360,
 'tn': 528,
 'fp': 123,
 'fn': 151,
 'auroc': 0.8264539576325448,
 'auprc': 0.8323302900851539,
 'eval_loss': 0.896010353140635}

In [23]:
# you can also use sklearn's neat classification report to get more metrics
from sklearn.metrics import classification_report

In [17]:
preds, probs = model.predict(list(test_df['Tweet'].values))
# preds = le.inverse_transform(preds)

print(classification_report(test_df['labels'], preds))

  0%|          | 0/1162 [00:00<?, ?it/s]

  0%|          | 0/146 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.78      0.81      0.79       651
           1       0.75      0.70      0.72       511

    accuracy                           0.76      1162
   macro avg       0.76      0.76      0.76      1162
weighted avg       0.76      0.76      0.76      1162



In [18]:
probs

array([[-3.00390625,  2.765625  ],
       [ 2.70507812, -2.28320312],
       [ 1.73632812, -1.59765625],
       ...,
       [ 2.83007812, -2.34375   ],
       [-2.74609375,  2.48242188],
       [-2.74609375,  2.55078125]])

We now repeat the same process with the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html).

In [19]:
!pip3 install transformers

In [20]:
! pip install -U accelerate
! pip install -U transformers

We will again use DistilBERT.

In [21]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

We will set some of the configurations

In [22]:
model_name = 'distilbert-base-uncased'
device_name = 'cuda'

# This is the maximum number of tokens in any document; the rest will be truncated.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'output_hf'

We will reuse the train-test splits we created for simpletransformers, but change the data structure slightly.

In [23]:
train_texts = train_df['Tweet']#.values
train_labels = train_df['labels']#.values

test_texts = test_df['Tweet']#.values
test_labels = test_df['labels']#.values

Compared to simpletransformers, we get a closer look at what happens 'under the hood' with huggingface. We will see the transformation of the text better --- each tweet will be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens.

The tokens will be separated into "word pieces" using the transformers tokenizers ('DistilBertTokenizerFast' in this case to match the DistiBERT model). And some special tokens will also be added such as **CLS** (start token of every tweet) and **SEP** (separator between each sentence {not tweet}):

In [24]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

We now encode our texts using the tokenizer.

In [25]:
from datasets import Dataset

train_df = Dataset.from_pandas(train_df)
test_df = Dataset.from_pandas(test_df)

def tokenize_function(examples):
  return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_train_df = train_df.map(tokenize_function, batched=True)
tokenized_test_df = test_df.map(tokenize_function, batched=True)

Map:   0%|          | 0/4644 [00:00<?, ? examples/s]

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

We now load the DistilBERT model and specify that it should use the GPU.

In [26]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_)).to(device_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


As we did with simpletransformers, we now set the training parameters, i.e., the number of epochs.

In [27]:
training_args = TrainingArguments(
    num_train_epochs=3,              # total number of training epochs
    output_dir='./results',          # output directory
    report_to='none'
)

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [28]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

In [29]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_train_df,         # training dataset
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [30]:
tokenized_train_df

Dataset({
    features: ['Tweet', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 4644
})

Time to finally fine-tune!

In [31]:
trainer.train()

Step,Training Loss
500,0.522200
1000,0.346300
1500,0.202100


Step,Training Loss
500,0.522200
1000,0.346300
1500,0.202100


TrainOutput(global_step=1743, training_loss=0.32654166207940016, metrics={'train_runtime': 695.1883, 'train_samples_per_second': 20.041, 'train_steps_per_second': 2.507, 'total_flos': 1845535798075392.0, 'train_loss': 0.32654166207940016, 'epoch': 3.0})

<br><br>

## **Save fine-tuned model**

The following cell will save the model and its configuration files to a directory in Colab. To preserve this model for future use, you should download the model to your computer.

In [32]:
trainer.save_model(cached_model_directory_name)

(Optional) If you've already fine-tuned and saved the model, you can reload it using the following line. You don't have to run fine-tuning every time you want to evaluate.

In [33]:
# trainer = DistilBertForSequenceClassification.from_pretrained(cached_model_directory_name)

We can now evaluate the model by predicting the labels for the test set.

In [34]:
predicted_results = trainer.predict(tokenized_test_df)

In [40]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels[0:5]

[1, 0, 0, 0, 0]

In [39]:
print(classification_report(tokenized_test_df['labels'],
                            predicted_labels))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82       651
           1       0.78      0.73      0.75       511

    accuracy                           0.79      1162
   macro avg       0.79      0.78      0.78      1162
weighted avg       0.79      0.79      0.79      1162



Now, it's your turn. Try the following:
- train a BERT  model instead of a distilBERT model
- try to model a multiclass classification problem. You can either find a dataset you'd like to try out or try with the PStance dataset.

Hint: Try to get all the datasets listed in https://drive.google.com/drive/folders/1so8lY1XKpnhUtTvb15edEz6aeHt7CSuh and classify the target of a tweet.

1. using a BERT model for simpletransformers

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=3, output_dir='output_st', overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-uncased", args=model_args
)

# Train the model
model.train_model(train_df)

1.2 BERT in HuggingFace

In [41]:
from transformers import BertTokenizerFast, BertForSequenceClassification

In [42]:
model_name = 'bert-base-uncased'
device_name = 'cuda'

# This is the maximum number of tokens in any document; the rest will be truncated.
max_length = 512

# This is the name of the directory where we'll save our model. You can name it whatever you want.
cached_model_directory_name = 'output_hf'

In [43]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)

We now encode our texts using the tokenizer.

In [44]:
def tokenize_function(examples):
  return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_train_df = train_df.map(tokenize_function, batched=True)
tokenized_test_df = test_df.map(tokenize_function, batched=True)

Map:   0%|          | 0/4644 [00:00<?, ? examples/s]

Map:   0%|          | 0/1162 [00:00<?, ? examples/s]

In [45]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_)).to(device_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
training_args = TrainingArguments(
    num_train_epochs=3,              # total number of training epochs
    output_dir='./results',          # output directory
    report_to='none'
)

In [47]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_train_df,         # training dataset
    compute_metrics=compute_metrics      # our custom evaluation function
)

In [48]:
trainer.train()

Step,Training Loss
500,0.544400
1000,0.373200
1500,0.247600


Step,Training Loss
500,0.544400
1000,0.373200
1500,0.247600


TrainOutput(global_step=1743, training_loss=0.3573979166001338, metrics={'train_runtime': 1347.5857, 'train_samples_per_second': 10.338, 'train_steps_per_second': 1.293, 'total_flos': 3665663223275520.0, 'train_loss': 0.3573979166001338, 'epoch': 3.0})

## 2. Multiclass classification

First, let's stack the PStance datasets across different targets into one dataset with different targets. The targets will be our final label to predict.

In [12]:
df_trump = pd.read_csv('raw_train_trump.csv')
df_biden = pd.read_csv('raw_train_biden.csv')
df_sanders = pd.read_csv('raw_train_bernie.csv')
all_data = pd.concat([df_trump,df_biden,df_sanders],ignore_index=True)
all_data.head()

,Tweet,Target,Stance
0,You COWARD... you should have been at THE WHIT...,Donald Trump,AGAINST
1,Cool. I trust he requested the same records fo...,Donald Trump,AGAINST
2,Off your meds again Donnie? You are surrounded...,Donald Trump,AGAINST
3,That has to happen anyway. But his profits wil...,Donald Trump,AGAINST
4,This is so stupid. Unmasking lets them know wh...,Donald Trump,AGAINST


In [13]:
all_data.groupby('Target').size()

Target
Bernie Sanders    5056
Donald Trump      6362
Joe Biden         5806
dtype: int64

Repeat the same steps as earlier for splitting this into train and test splits, but this time stratify on 'Target', not 'Stance'

In [14]:
train_df, test_df = train_test_split(all_data, stratify=all_data['Target'], test_size=0.2)

Let's encode the labels again.

In [15]:
le = LabelEncoder()
le.fit(train_df['Target'])
train_df['labels'] = le.transform(train_df['Target'])
test_df['labels'] = le.transform(test_df['Target'])

In [16]:
# to see which number was mapped to which class:
list(le.inverse_transform([0, 1, 2]))

['Bernie Sanders', 'Donald Trump', 'Joe Biden']

The rest is exacty the same.

In [17]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=3, output_dir='output_st',
                                overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel(
    "distilbert", "distilbert-base-uncased",
    num_labels=len(le.classes_),
    args=model_args
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
train_df = train_df[['Tweet', 'labels']]
test_df = test_df[['Tweet', 'labels']]

In [19]:
# let's take a smaple of the training data so that it runs faster
train_df = train_df.sample(4000)

In [20]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/4000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/500 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/500 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/500 [00:00<?, ?it/s]

(1500, 0.7545285911560059)

After training our model, we can use it to make predictions for unlabeled datapoints to classify the target of the tweet.

In [21]:
anti_biden_tweet = "Ugh, this was true yesterday and it's also true now: Biden is an idiot"
predictions, raw_outputs = model.predict([anti_biden_tweet])
le.inverse_transform(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array(['Joe Biden'], dtype=object)

We can also use the held-out test set to quantitatively evaluate our model.

In [22]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3445 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/431 [00:00<?, ?it/s]

{'mcc': 0.3946119790990069, 'eval_loss': 1.1035260733761534}

In [24]:
preds, probs = model.predict(list(test_df['Tweet'].values))
# preds = le.inverse_transform(preds)

print(classification_report(test_df['labels'], preds))

  0%|          | 0/3445 [00:00<?, ?it/s]

  0%|          | 0/431 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.60      0.59      0.59      1011
           1       0.64      0.63      0.64      1273
           2       0.55      0.57      0.56      1161

    accuracy                           0.60      3445
   macro avg       0.60      0.60      0.60      3445
weighted avg       0.60      0.60      0.60      3445

